In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     

# lagirl 줄글로 된 사이즈표 크롤링

In [3]:
base_url = "https://www.lagirl.co.kr"

In [4]:
req = requests.get(base_url)
if req == 200 :
    soup = BeautifulSoup(req.content, 'html.parser')
else :
    req = requests.get(base_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup = BeautifulSoup(req.content, 'html.parser')

In [6]:
category_list1 = soup.find('div',{'class':'xans-element- xans-layout xans-layout-category'})
#getcategory가 class와 붙어있는 것처럼 보이지만 중간에 공백으로 띄어쓰기가 되어있기 때문에
#class를 class로 사용한다

In [10]:
category_ls = category_list1.find_all('a')

In [17]:
category_ls[0]['href']

'/product/list.html?cate_no=104'

In [20]:
category_url = []
category_name = []
for i in range(len(category_ls)):
    category_ls2 = base_url + category_ls[i]['href']
    category_nm = category_ls[i].text
    category_url.append(category_ls2)
    category_name.append(category_nm)

In [24]:
category_list_url = category_url[5:9]

In [25]:
start_url = category_list_url[0]
page_url = category_list_url[0] + '&page='+'{}'

In [27]:
req2 = requests.get(start_url)
if req2 == 200 :
    soup2 = BeautifulSoup(req2.content, 'html.parser')
else :
    req2 = requests.get(start_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup2 = BeautifulSoup(req2.content, 'html.parser')

page1_a = soup2.find('div',{'class':'xans-element- xans-product xans-product-normalpaging ec-base-paginate'})
page2_a = page1_a.find_all('li',{'class','xans-record-'})



In [30]:
page1_b = page1_a.find_all('a')[-1]['href'] #가장 마지막 페이지로 가는 url
page2_b = start_url + page1_b
req3 = requests.get(page2_b)
if req3 == 200 :
    soup3 = BeautifulSoup(req3.content, 'html.parser')
else :
    req3 = requests.get(page2_b, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup3 = BeautifulSoup(req3.content, 'html.parser')

page3_b = soup3.find('div',{'class':'xans-element- xans-product xans-product-normalpaging ec-base-paginate'})
page4_b = page3_b.find_all('li',{'class','xans-record-'})

page5 = len(page2_a) + len(page4_b)

In [31]:
page5

7

In [32]:
for i in range(1,page5+1):
    req4 = requests.get(page_url.format(str(i)))
    if req4 == 200 :
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    else :
        req4 = requests.get(page_url.format(str(i)), headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    
    product_1 = soup4.find('div',{'class':'xans-element- xans-product xans-product-listnormal ec-base-product'})
    product_2 = product_1.find_all('li',{'class':'item xans-record-'})

In [56]:
for i in range(len(product_2)):
        name0 = product_2[i].find('p',{'class':'name'})
        name = name0.find_all('span')[2].text

        url = base_url + name0.find('a')['href']
        
        price0 = product_2[i].find('ul',{'class':'xans-element- xans-product xans-product-listitem spec'})
        price1 = price0.find_all('li',{'class':'xans-record-'})
        
        if len(price1)>2:
            price = price1[1]('span')[1].text
        else:
            price = price1[0]('span')[1].text
        
        req5 = requests.get(url)
        if req5 == 200 :
            soup5 = BeautifulSoup(req5.content, 'html.parser')
        else :
            req5 = requests.get(url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
            soup5 = BeautifulSoup(req5.content, 'html.parser')

        img0 = soup5.find('div',{'class':'thumbnail'})
        img = img0.find('img')['src']

        insert_data = pd.DataFrame({"date" : datetime.today().strftime("%Y-%m-%d"),
                                    "image" : [img],
                                    "name" : [name],
                                    "price" : [price],
                                    "link" : [url]})

In [2]:
data = pd.DataFrame({"date" : [],
                    "image" : [],
                    "name" : [],
                    "price" : [],
                    "link" : []})

base_url = "https://www.lagirl.co.kr"

req = requests.get(base_url)
if req == 200 :
    soup = BeautifulSoup(req.content, 'html.parser')
else :
    req = requests.get(base_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup = BeautifulSoup(req.content, 'html.parser')

category_list1 = soup.find('div',{'class':'xans-element- xans-layout xans-layout-category'})
#getcategory가 class와 붙어있는 것처럼 보이지만 중간에 공백으로 띄어쓰기가 되어있기 때문에
#class를 class로 사용한다
category_ls = category_list1.find_all('a')
category_url = []
category_name = []
for i in range(len(category_ls)):
    category_ls2 = base_url + category_ls[i]['href']
    category_nm = category_ls[i].text
    category_url.append(category_ls2)
    category_name.append(category_nm)
    
category_list_url = category_url[5:9]

start_url = category_list_url[0]
page_url = category_list_url[0] + '&page='+'{}'

req2 = requests.get(start_url)
if req2 == 200 :
    soup2 = BeautifulSoup(req2.content, 'html.parser')
else :
    req2 = requests.get(start_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup2 = BeautifulSoup(req2.content, 'html.parser')

page1_a = soup2.find('div',{'class':'xans-element- xans-product xans-product-normalpaging ec-base-paginate'})
page2_a = page1_a.find_all('li',{'class','xans-record-'})
page1_b = page1_a.find_all('a')[-1]['href'] #가장 마지막 페이지로 가는 url
page2_b = start_url + page1_b

req3 = requests.get(page2_b)
if req3 == 200 :
    soup3 = BeautifulSoup(req3.content, 'html.parser')
else :
    req3 = requests.get(page2_b, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup3 = BeautifulSoup(req3.content, 'html.parser')

page3_b = soup3.find('div',{'class':'xans-element- xans-product xans-product-normalpaging ec-base-paginate'})
page4_b = page3_b.find_all('li',{'class','xans-record-'})

page5 = len(page2_a) + len(page4_b)

for i in range(1,page5+1):
    req4 = requests.get(page_url.format(str(i)))
    if req4 == 200 :
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    else :
        req4 = requests.get(page_url.format(str(i)), headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    
    product_1 = soup4.find('div',{'class':'xans-element- xans-product xans-product-listnormal ec-base-product'})
    product_2 = product_1.find_all('li',{'class':'item xans-record-'})
    
    for i in range(len(product_2)):
        name0 = product_2[i].find('p',{'class':'name'})
        name = name0.find_all('span')[2].text

        url = base_url + name0.find('a')['href']
        
        price0 = product_2[i].find('ul',{'class':'xans-element- xans-product xans-product-listitem spec'})
        price1 = price0.find_all('li',{'class':'xans-record-'})
        
        if len(price1)>2:
            price = price1[1]('span')[1].text
        else:
            price = price1[0]('span')[1].text
        
        req5 = requests.get(url)
        if req5 == 200 :
            soup5 = BeautifulSoup(req5.content, 'html.parser')
        else :
            req5 = requests.get(url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
            soup5 = BeautifulSoup(req5.content, 'html.parser')

        img0 = soup5.find('div',{'class':'thumbnail'})
        img = img0.find('img')['src']

        insert_data = pd.DataFrame({"date" : datetime.today().strftime("%Y-%m-%d"),
                                    "image" : [img],
                                    "name" : [name],
                                    "price" : [price],
                                    "link" : [url]})
        
        data = data.append(insert_data)
            

data = data.drop_duplicates(subset='image', keep='last') # 중복된 부분이 있다면 가장 최근것으로 대체
data.index = range(len(data))

In [312]:
data

,date,image,name,price,link
0,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/0...,후드아노락-jp,"56,100won",https://www.lagirl.co.kr/product/detail.html?p...
1,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/1...,라인케이프-jk,"99,800won",https://www.lagirl.co.kr/product/detail.html?p...
2,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/5...,센슈얼체크-jk,"65,600won",https://www.lagirl.co.kr/product/detail.html?p...
3,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/f...,모노대디-jk,"68,400won",https://www.lagirl.co.kr/product/detail.html?p...
4,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/9...,유닉볼륨청-jk,"37,100won",https://www.lagirl.co.kr/product/detail.html?p...
5,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/c...,퍼플데님-jk,"59,000won",https://www.lagirl.co.kr/product/detail.html?p...
6,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/6...,투웨이패드-jk,"115,000won",https://www.lagirl.co.kr/product/detail.html?p...
7,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/b...,버튼스트랩-jk,"75,000won",https://www.lagirl.co.kr/product/detail.html?p...
8,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/1...,스티트렌치-bby,"109,000won",https://www.lagirl.co.kr/product/detail.html?p...
9,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/d...,체크벌룬-jk,"39,000won",https://www.lagirl.co.kr/product/detail.html?p...


# 사이즈표 뽑기

In [313]:
prds = []
index = []
names = []
for i in range(len(data)) :
    prd = data['link'][i]
    ind = data.index[i]
    name = data['name'][i]
    if prd not in prds : # 중복되는 상품이 없도록 url을 수집한다.
        prds.append(prd) 
        index.append(ind) # 해당 상품에 대한 인덱스 정보가 있어야 이후에 옷 이미지와 매칭시킬 수 있으므로 인덱스 정보까지 포함한다.
        names.append(name)

In [ ]:
df2 = pd.DataFrame({"name":[],
                    "size" : [],
                    "어깨" : [],
                    "소매" : [],
                    "암홀" : [],
                    "가슴" : [],
                   "총" : []})
for web in range(len(prds)):
    if 'set' not in names[web]:
        try:
            test_item = prds[web]

            req6 = requests.get(test_item)
            if req6 == 200 :
                soup6 = BeautifulSoup(req6.content, 'html.parser')
            else :
                req6 = requests.get(test_item, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
                soup6 = BeautifulSoup(req6.content, 'html.parser')

            size0 = soup6.find('div',{'class':'cont'}).find_all('p',{'style':['text-align: center;','text-align: center; ']})
            if size0 !=[]:
                pass
            else : 
                size0=soup6.find('div',{'class':'cont'}).find_all('span',{'style':'font-family: 돋움,dotum;'})

            text0 = []
            for i in range(len(size0)):
                tt  = size0[i].text
                text0.append(tt)
            text0=set(text0)

            matchers = ['단위'] #단위와 어깨/암홀로 한 차이가 약 30상품(에러가 30개)차이가 남
            matching = [s for s in text0 if any(xs in s for xs in matchers)]

            size_t = matching[0].split()    
            size_nm = size_t[0]
            size_t2 = size_t[1:-1]

            size_col = ['size']
            size_row = [size_nm]
            for i in range(len(size_t2)):
                match = re.match(r"([가-힣]+)([0-9].+)", size_t2[i], re.I)
                if match:
                    items = match.groups()
                size_col.append(items[0])
                size_row.append(items[1])

            df_iframe = pd.DataFrame( columns = size_col)
            df = df_iframe.append(pd.Series(size_row, index=size_col), ignore_index=True)
            df["name"] = names[web]

            df2 = df2.append(df)
            #사이즈가 있는 아우터 조건문

        except:
            pass
    
df2["category"] = "Outer"
df2.index = range(len(df2))

In [ ]:
for web in range(len(prds)):
    if 'set' not in names[web]:
        try:
            test_item = prds[web]

            req6 = requests.get(test_item)
            if req6 == 200 :
                soup6 = BeautifulSoup(req6.content, 'html.parser')
            else :
                req6 = requests.get(test_item, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
                soup6 = BeautifulSoup(req6.content, 'html.parser')

            size0 = soup6.find('div',{'class':'cont'}).find_all('p',{'style':['text-align: center;','text-align: center; ']})
            if size0 !=[]:
                pass
            else : 
                size0=soup6.find('div',{'class':'cont'}).find_all('span',{'style':'font-family: 돋움,dotum;'})

            text0 = []
            for i in range(len(size0)):
                tt  = size0[i].text
                text0.append(tt)
            text0=set(text0)

            matchers = ['어깨','암홀']
            matching = [s for s in text0 if any(xs in s for xs in matchers)]
            matching

            if '단위' not in matching[0]:
                size_t = []
                for i in range(len(matching)):
                    size_t0 = matching[i].split()
                    size_t.append(size_t0)

                size_nm = []
                size_t2 = []
                for i in range(len(size_t)):
                    size_nm.append(size_t[i][0])
                    size_t2.append(size_t[i][1:])

                size_row2 = []
                size_col = ['size']
                for i in range(len(size_nm)):
                    size_row = [size_nm[i]]
                    for j in range(len(size_t2[0])):
                        match = re.match(r"([가-힣]+)([0-9].+)", size_t2[i][j], re.I)
                        if match:
                            items=match.groups()
                        size_row.append(items[1])
                        size_col.append(items[0])
                    size_row2.append(size_row)

                size_col2=size_col[0:len(size_t[0])] 

                data_f = pd.DataFrame(columns = size_col2) #data_f = pd.Dataframe()로 놓는 것이 좋음
                df_iframe = pd.DataFrame( columns = size_col2)
                for i in range(len(size_row2)):
                    df = df_iframe.append(pd.Series(size_row2[i], index=size_col2), ignore_index=True)
                    df["name"] = names[web]
                    df["category"] = 'outer'
                    data_f = data_f.append(df)
                    
                df2 = df2.append(data_f)
        except:
            pass
df2.index = range(len(df2))

In [362]:
df2_outer = df2.where((pd.notnull(df2)), None)
df2_outer #아우터를 뽑고 사이즈가 있는 아우터는 따로 뽑아서 append해야할 듯

,category,name,size,가슴,끈,뒷트임,밑단트임,벨트,벨트총,소매,소매트임,스트랩,스트랩단면,스트랩총,암홀,어깨,총,트임,허리
0,Outer,후드아노락-jp,(F44-66),60.5,None,None,None,None,None,61,None,None,None,None,29,46.5,53.5,None,None
1,Outer,센슈얼체크-jk,(F44-66),60,None,None,None,None,None,60,None,None,None,None,28,44.5,78,26,None
2,Outer,모노대디-jk,(F44-66),55,None,None,None,None,None,63,None,None,None,None,23,39,75.5,None,None
3,Outer,퍼플데님-jk,(F44-66),53,None,None,None,None,None,48,None,None,None,None,22.5,63.5,55.5,None,None
4,Outer,투웨이패드-jk,(F44-66),54,None,None,None,None,None,60.5,None,None,None,None,27,49,74(44),None,None
5,Outer,버튼스트랩-jk,(F44-66),40,None,None,None,None,None,58.5,None,None,53,None,24,38,80,None,None
6,Outer,스티트렌치-bby,(F44-66),48.5,None,None,None,None,None,53,None,175,None,None,26,50,117.5,None,None
7,Outer,체크벌룬-jk,(F44-66),45.5,None,None,None,None,None,62,None,158,None,None,24,36,66,None,None
8,Outer,베이직연청-jk,(F44-66),55,None,None,None,None,None,60,None,152,None,None,24.5,42,80,None,None
9,Outer,델리롱더블-ct,(F44-66),49,None,None,None,None,None,64,None,352,None,None,26,43,106,47,None


In [363]:
df2_outer.to_json(orient='records')

'[{"category":"Outer","name":"\\ud6c4\\ub4dc\\uc544\\ub178\\ub77d-jp","size":"(F44-66)","\\uac00\\uc2b4":"60.5","\\ub048":null,"\\ub4b7\\ud2b8\\uc784":null,"\\ubc11\\ub2e8\\ud2b8\\uc784":null,"\\ubca8\\ud2b8":null,"\\ubca8\\ud2b8\\ucd1d":null,"\\uc18c\\ub9e4":"61","\\uc18c\\ub9e4\\ud2b8\\uc784":null,"\\uc2a4\\ud2b8\\ub7a9":null,"\\uc2a4\\ud2b8\\ub7a9\\ub2e8\\uba74":null,"\\uc2a4\\ud2b8\\ub7a9\\ucd1d":null,"\\uc554\\ud640":"29","\\uc5b4\\uae68":"46.5","\\ucd1d":"53.5","\\ud2b8\\uc784":null,"\\ud5c8\\ub9ac":null},{"category":"Outer","name":"\\uc13c\\uc288\\uc5bc\\uccb4\\ud06c-jk","size":"(F44-66)","\\uac00\\uc2b4":"60","\\ub048":null,"\\ub4b7\\ud2b8\\uc784":null,"\\ubc11\\ub2e8\\ud2b8\\uc784":null,"\\ubca8\\ud2b8":null,"\\ubca8\\ud2b8\\ucd1d":null,"\\uc18c\\ub9e4":"60","\\uc18c\\ub9e4\\ud2b8\\uc784":null,"\\uc2a4\\ud2b8\\ub7a9":null,"\\uc2a4\\ud2b8\\ub7a9\\ub2e8\\uba74":null,"\\uc2a4\\ud2b8\\ub7a9\\ucd1d":null,"\\uc554\\ud640":"28","\\uc5b4\\uae68":"44.5","\\ucd1d":"78","\\ud2b8\\uc784":

# 사이즈표 뽑기 위한 practice

In [127]:
test_item = prds[7]

In [128]:
req6 = requests.get(test_item)
if req6 == 200 :
    soup6 = BeautifulSoup(req6.content, 'html.parser')
else :
    req6 = requests.get(test_item, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
    soup6 = BeautifulSoup(req6.content, 'html.parser')

In [132]:
size0 = soup6.find('div',{'class':'cont'}).find_all('p',{'style':['text-align: center;','text-align: center; ']})

In [133]:
text0 = []
for i in range(len(size0)):
    tt  = size0[i].text
    text0.append(tt)


In [134]:
text0

['COMMENT',
 '트렌디한 디자인의 자켓 소개해 드려요',
 '넉넉한 핏의 디자인으로 나와 누구나 착용!',
 '숄더 패드가 함께 부착 되어 라인을 살려주며s',
 '유니크한 단추 스트랩이 함께 부착 되어있어',
 '다양한 핏 조절은 물론 포인트를 줄수 있어요',
 '컬러는 베이지 체크 두가지 준비해 두었어요',
 '그럼 상세 사이즈 참고하시어 굿 초이스 하세요',
 '',
 'FABRIC',
 '베이지-폴리97% 스판3%',
 '체크-폴리90% 레이온8% 스판2%',
 '안감X / 신축성X / 비침X / 두께감△ / 촉감△ ',
 '',
 'COLOR',
 '베이지(BEIGE),체크(CHECK)',
 '',
 'SIZE',
 '(F44-66) 어깨38 소매58.5\xa0 암홀24 가슴40 스트랩단면53 총80 (단위:CM/단면)',
 'JIHYE\xa0 162CM/43KG/44반~55SIZE(S)/25INCH/230MM',
 'YEONSU 165CM/45KG/44~55SIZE(sS)/25~26INCH/235MM',
 '(사이즈는 재는 방법에 따라 2~4cm 차이가 있을 수 있습니다.)',
 '',
 'WASHING TIP',
 '드라이크리닝○ / 손세탁X / 짜면 안돼요! / 세탁기X / 다림질○']

In [135]:
matchers = ['단위']

matching = [s for s in text0 if any(xs in s for xs in matchers)]
            
matching

['(F44-66) 어깨38 소매58.5\xa0 암홀24 가슴40 스트랩단면53 총80 (단위:CM/단면)']

In [136]:
matching

['(F44-66) 어깨38 소매58.5\xa0 암홀24 가슴40 스트랩단면53 총80 (단위:CM/단면)']

In [137]:
size_t = matching[0].split()

In [138]:
size_nm = size_t[0]
size_t2 = size_t[1:-1]


In [139]:
size_t2 = size_t[1:-1]

In [140]:
size_t2

['어깨38', '소매58.5', '암홀24', '가슴40', '스트랩단면53', '총80']

In [147]:
size_col = ['size']
size_row = [size_nm]
for i in range(len(size_t2)):
    match = re.match(r"([가-힣]+)([0-9.]+)", size_t2[i], re.I)
    if match:
        items = match.groups()
    size_col.append(items[0])
    size_row.append(items[1])

In [148]:
size_col

['size', '어깨', '소매', '암홀', '가슴', '스트랩단면', '총']

In [149]:
size_row

['(F44-66)', '38', '58.5', '24', '40', '53', '80']

In [126]:
df_iframe = pd.DataFrame( columns = size_col)
df = df_iframe.append(pd.Series(size_row, index=size_col), ignore_index=True)
df["name"] = 'dd'
            


In [127]:
df

,size,어깨,소매,암홀,가슴,총,name
0,(F44-66),47,57.5,39,53,56.5,dd


In [ ]:
df2 = df2.append(df)
df2["category"] = "Outer"

# lagirl pants

In [4]:
data2 = pd.DataFrame({"date" : [],
                    "image" : [],
                    "name" : [],
                    "price" : [],
                    "link" : []})

base_url = "https://www.lagirl.co.kr"

req = requests.get(base_url)
if req == 200 :
    soup = BeautifulSoup(req.content, 'html.parser')
else :
    req = requests.get(base_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup = BeautifulSoup(req.content, 'html.parser')

category_list1 = soup.find('div',{'class':'xans-element- xans-layout xans-layout-category'})
#getcategory가 class와 붙어있는 것처럼 보이지만 중간에 공백으로 띄어쓰기가 되어있기 때문에
#class를 class로 사용한다
category_ls = category_list1.find_all('a')
category_url = []
category_name = []
for i in range(len(category_ls)):
    category_ls2 = base_url + category_ls[i]['href']
    category_nm = category_ls[i].text
    category_url.append(category_ls2)
    category_name.append(category_nm)
    
category_list_url = category_url[5:9]

start_url = category_list_url[2]
page_url = category_list_url[2] + '&page='+'{}'

req2 = requests.get(start_url)
if req2 == 200 :
    soup2 = BeautifulSoup(req2.content, 'html.parser')
else :
    req2 = requests.get(start_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup2 = BeautifulSoup(req2.content, 'html.parser')

page1_a = soup2.find('div',{'class':'xans-element- xans-product xans-product-normalpaging ec-base-paginate'})
page2_a = page1_a.find_all('li',{'class','xans-record-'})

page1_b = page1_a.find_all('a')[-1]['href'] #가장 마지막 페이지로 가는 url
page2_b = start_url + page1_b

req3 = requests.get(page2_b)
if req3 == 200 :
    soup3 = BeautifulSoup(req3.content, 'html.parser')
else :
    req3 = requests.get(page2_b, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup3 = BeautifulSoup(req3.content, 'html.parser')

page3_b = soup3.find('div',{'class':'xans-element- xans-product xans-product-normalpaging ec-base-paginate'})
page4_b = page3_b.find_all('li',{'class','xans-record-'})

if 1<=int(page1_b[-1])<6:
    page5 = 5 + len(page2_a) + len(page4_b)
else:
    page5 = len(page2_a) + len(page4_b)

for i in range(1,page5+1):
    req4 = requests.get(page_url.format(str(i)))
    if req4 == 200 :
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    else :
        req4 = requests.get(page_url.format(str(i)), headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    
    product_1 = soup4.find('div',{'class':'xans-element- xans-product xans-product-listnormal ec-base-product'})
    product_2 = product_1.find_all('li',{'class':'item xans-record-'})
    
    for i in range(len(product_2)):
        name0 = product_2[i].find('p',{'class':'name'})
        name = name0.find_all('span')[2].text

        url = base_url + name0.find('a')['href']
        
        price0 = product_2[i].find('ul',{'class':'xans-element- xans-product xans-product-listitem spec'})
        price1 = price0.find_all('li',{'class':'xans-record-'})
        
        if len(price1)>2:
            price = price1[1]('span')[1].text
        else:
            price = price1[0]('span')[1].text
        
        req5 = requests.get(url)
        if req5 == 200 :
            soup5 = BeautifulSoup(req5.content, 'html.parser')
        else :
            req5 = requests.get(url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
            soup5 = BeautifulSoup(req5.content, 'html.parser')

        img0 = soup5.find('div',{'class':'thumbnail'})
        img = img0.find('img')['src']

        insert_data = pd.DataFrame({"date" : datetime.today().strftime("%Y-%m-%d"),
                                    "image" : [img],
                                    "name" : [name],
                                    "price" : [price],
                                    "link" : [url]})
        
        data2 = data2.append(insert_data)
            

data2 = data2.drop_duplicates(subset='image', keep='last') # 중복된 부분이 있다면 가장 최근것으로 대체
data2.index = range(len(data2))

In [8]:
data2

,date,image,name,price,link
0,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/d...,부츠라인데님-pt,"46,600won",https://www.lagirl.co.kr/product/detail.html?p...
1,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/a...,비비미디-sk,"33,300won",https://www.lagirl.co.kr/product/detail.html?p...
2,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/0...,연청와이드핏-pt,"43,700won",https://www.lagirl.co.kr/product/detail.html?p...
3,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/d...,지브라밴딩-pt,"45,600won",https://www.lagirl.co.kr/product/detail.html?p...
4,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/4...,점프돌청-j/s,"27,600won",https://www.lagirl.co.kr/product/detail.html?p...
5,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/2...,하프앤데님-shorts,"25,700won",https://www.lagirl.co.kr/product/detail.html?p...
6,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/5...,플리츠밴딩-pt,"37,100won",https://www.lagirl.co.kr/product/detail.html?p...
7,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/2...,후드숏-set,"37,100won",https://www.lagirl.co.kr/product/detail.html?p...
8,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/c...,기본레더-sk,"35,000won",https://www.lagirl.co.kr/product/detail.html?p...
9,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/c...,하프앤숏-shorts,"42,000won",https://www.lagirl.co.kr/product/detail.html?p...


In [150]:
prds = []
index = []
names = []
for i in range(len(data)) :
    prd = data['link'][i]
    ind = data.index[i]
    name = data['name'][i]
    if prd not in prds : # 중복되는 상품이 없도록 url을 수집한다.
        prds.append(prd) 
        index.append(ind) # 해당 상품에 대한 인덱스 정보가 있어야 이후에 옷 이미지와 매칭시킬 수 있으므로 인덱스 정보까지 포함한다.
        names.append(name)

In [ ]:
#일단 힙과 허벅지로 뽑았을 때 설명글까지 뽑는 경우에는 데이터로 불러들이지 않는다 조건문을 달 수 있으나 보류
#이게 진짜임
df2 = pd.DataFrame({"name":[],
                    "size" : [],
                    "허리" : [],
                    "힙" : [],
                    "밑위" : [],
                    "허벅지" : [],
                   "밑단" : [],
                   "트임" : [],
                   "총":[]})
for web in range(0,50):
    if 'set' not in names[web]:
        try:
            test_item = prds[web]
            req6 = requests.get(test_item)
            if req6 == 200 :
                soup6 = BeautifulSoup(req6.content, 'html.parser')
            else :
                req6 = requests.get(test_item, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
                soup6 = BeautifulSoup(req6.content, 'html.parser')
            size0 = soup6.find('div',{'class':'cont'}).find_all('p',{'style':['text-align: center;','text-align: center; ']})
            if size0 !=[]:
                pass
            else :
                size0=soup6.find('div',{'class':'cont'}).find_all('span',{'style':'font-family: 돋움,dotum;'})
                
            text0 = []
            for i in range(len(size0)):
                tt  = size0[i].text
                text0.append(tt)
            
            text0 = set(text0)#리스트 중복제거
            
            matchers = ['힙','허벅지']

            matching = [s for s in text0 if any(xs in s for xs in matchers)]

            matching
            
            if '단위' in matching[0]: #if문사용 - 프리사이즈 - 해당 조건문 제대로 되는지 확인해야함 
            #프리사이즈가 데이터에 삽입되지가 않음...
                size_t = matching[0].split()
                size_nm = size_t[0]
                if size_t[-1] == '(단위:CM/단면)':
                    size_t2 = size_t[1:-1]
                else:
                    size_t2 = size_t[1:]

                size_col = ['size']
                size_row = [size_nm]
                for i in range(len(size_t2)):
                    match = re.match(r"([^a-zA-Z0-9]+)([0-9].+)", size_t2[i], re.I)
                    if match:
                        items = match.groups()
                    size_col.append(items[0])
                    size_row.append(items[1])

                df_iframe = pd.DataFrame( columns = size_col)
                df = df_iframe.append(pd.Series(size_row, index=size_col), ignore_index=True)
                df["name"] = names[web]

                df2 = df2.append(df)
            
            if '단위' not in matching[0]:
                size_t = []
                for i in range(len(matching)):
                    size_t0 = matching[i].split()
                    size_t.append(size_t0)

                size_nm = []
                size_t2 = []
                for i in range(len(size_t)):
                    size_nm.append(size_t[i][0])
                    size_t2.append(size_t[i][1:])

                size_row2 = []
                size_col = ['size']
                for i in range(len(size_nm)):
                    size_row = [size_nm[i]]
                    for j in range(len(size_t2[0])):
                        match = re.match(r"([가-힣]+)([0-9].+)", size_t2[i][j], re.I)
                        if match:
                            items=match.groups()
                        size_row.append(items[1])
                        size_col.append(items[0])
                    size_row2.append(size_row)

                size_col2=size_col[0:len(size_t[0])]   

                data_f = pd.DataFrame(columns = size_col2)
                df_iframe = pd.DataFrame( columns = size_col2)
                for i in range(len(size_row2)):
                    df = df_iframe.append(pd.Series(size_row2[i], index=size_col2), ignore_index=True)
                    df["name"] = names[web]
                    data_f = data_f.append(df)

                df2 = df2.append(data_f)
            
        except:
            pass

In [ ]:
df2_json = df2.where((pd.notnull(df2)),None)

In [360]:
df2_json

,name,size,가슴,뒷트임,밑단,밑위,암홀(조절가능),어깨,총,트임,허리,허리(뒷밴딩),허리(밴딩),허리(조절가능),허벅지,힙
0,부츠라인데님-pt,S,None,None,23.5,25,None,None,97.5,None,32,None,None,None,22.5,38
0,부츠라인데님-pt,M,None,None,24.5,26,None,None,98.5,None,35,None,None,None,24.5,39
0,비비미디-sk,(F44-66),None,None,None,None,None,None,70,19.5,None,None,30,None,None,46
0,연청와이드핏-pt,S,None,None,20,29,None,None,97.5,None,32,None,None,None,30.5,47.5
0,연청와이드핏-pt,M,None,None,21,30,None,None,98.5,None,35,None,None,None,32.5,48.5
0,지브라밴딩-pt,(F44-66),None,None,33.5,37,None,None,104.5,None,28.5,None,None,None,32,39.5
0,점프돌청-j/s,(F44-66),43,None,20,None,42,28.5,130,None,None,None,None,None,None,45
0,플리츠밴딩-pt,(F44-66),None,None,25,31,None,None,95(단위:CM/단면),None,None,None,29,None,24,36
0,기본레더-sk,S,None,None,None,None,None,None,40,None,33,None,None,None,None,43
0,기본레더-sk,M,None,None,None,None,None,None,41,None,36,None,None,None,None,44


In [361]:
df2_json.to_json(orient='records')

'[{"name":"\\ubd80\\uce20\\ub77c\\uc778\\ub370\\ub2d8-pt","size":"S","\\uac00\\uc2b4":null,"\\ub4b7\\ud2b8\\uc784":null,"\\ubc11\\ub2e8":"23.5","\\ubc11\\uc704":"25","\\uc554\\ud640(\\uc870\\uc808\\uac00\\ub2a5)":null,"\\uc5b4\\uae68":null,"\\ucd1d":"97.5","\\ud2b8\\uc784":null,"\\ud5c8\\ub9ac":"32","\\ud5c8\\ub9ac(\\ub4b7\\ubc34\\ub529)":null,"\\ud5c8\\ub9ac(\\ubc34\\ub529)":null,"\\ud5c8\\ub9ac(\\uc870\\uc808\\uac00\\ub2a5)":null,"\\ud5c8\\ubc85\\uc9c0":"22.5","\\ud799":"38"},{"name":"\\ubd80\\uce20\\ub77c\\uc778\\ub370\\ub2d8-pt","size":"M","\\uac00\\uc2b4":null,"\\ub4b7\\ud2b8\\uc784":null,"\\ubc11\\ub2e8":"24.5","\\ubc11\\uc704":"26","\\uc554\\ud640(\\uc870\\uc808\\uac00\\ub2a5)":null,"\\uc5b4\\uae68":null,"\\ucd1d":"98.5","\\ud2b8\\uc784":null,"\\ud5c8\\ub9ac":"35","\\ud5c8\\ub9ac(\\ub4b7\\ubc34\\ub529)":null,"\\ud5c8\\ub9ac(\\ubc34\\ub529)":null,"\\ud5c8\\ub9ac(\\uc870\\uc808\\uac00\\ub2a5)":null,"\\ud5c8\\ubc85\\uc9c0":"24.5","\\ud799":"39"},{"name":"\\ube44\\ube44\\ubbf8\\ub51

In [147]:
matchers = ['허리','힙','허벅지']

matching = [s for s in text0 if any(xs in s for xs in matchers)]
            
matching

['S 허리32 힙40 밑위23 허벅지24.5 밑단21 트임19 총83',
 'M 허리35 힙41 밑위24 허벅지26.5 밑단22 트임19.5 총84',
 'L 허리38 힙42 밑위25 허벅지28.5 밑단23 트임20 총85']

In [64]:
size_t = []
for i in range(len(matching)):
    size_t0 = matching[i].split()
    size_t.append(size_t0)

In [70]:
size_nm = []
size_t2 = []
for i in range(len(size_t)):
    size_nm.append(size_t[i][0])
    size_t2.append(size_t[i][1:])
    
    


In [115]:
size_row2 = []
size_col = ['size']
for i in range(len(size_nm)):
    size_row = [size_nm[i]]
    for j in range(len(size_t2[0])):
        match = re.match(r"([가-힣]+)([0-9].+)", size_t2[i][j], re.I)
        if match:
            items=match.groups()
        size_row.append(items[1])
        size_col.append(items[0])
    size_row2.append(size_row)

size_col2=size_col[0:len(size_t[0])]    

In [117]:
size_row2[0]

['S', '32', '40', '23', '24.5', '21', '19', '83']

In [144]:
dataaa = pd.DataFrame(columns = size_col2)
df_iframe = pd.DataFrame( columns = size_col2)
for i in range(len(size_row2)):
    df = df_iframe.append(pd.Series(size_row2[i], index=size_col2), ignore_index=True)
    df["name"] = 'dd'
    dataaa = dataaa.append(df)



C:\Users\tlsal\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [146]:
dataaa

,name,size,밑단,밑위,총,트임,허리,허벅지,힙
0,dd,S,21,23,83,19,32,24.5,40
0,dd,M,22,24,84,19.5,35,26.5,41
0,dd,L,23,25,85,20,38,28.5,42


In [ ]:
size_nm = size_t[0]
size_t2 = size_t[1:]


In [51]:
size_col = ['size']# 얘는 따로 뽑아서 컬럼식으로만 만들면 됨
size_row = [size_nm]# 얘가 문제인데
for i in range(len(size_t2)):
    match = re.match(r"([가-힣]+)([0-9].+)", size_t2[i], re.I)
    if match:
        items = match.groups()
    size_col.append(items[0])
    size_row.append(items[1])

In [55]:
size_col

['size', '허리', '힙', '밑위', '허벅지', '밑단', '트임', '총']

In [56]:
df_iframe = pd.DataFrame( columns = size_col)
df = df_iframe.append(pd.Series(size_row, index=size_col), ignore_index=True)
df["name"] = 'dd'
            


In [57]:
df

,size,허리,힙,밑위,허벅지,밑단,트임,총,name
0,S,32,40,23,24.5,21,19,83,dd


In [61]:
'단위' in matching[0]

False

# freesize경우와 사이즈가 있는 경우를 따로 돌린 뒤
# 병합해야하나??

In [9]:
data2

,date,image,name,price,link
0,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/d...,부츠라인데님-pt,"46,600won",https://www.lagirl.co.kr/product/detail.html?p...
1,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/a...,비비미디-sk,"33,300won",https://www.lagirl.co.kr/product/detail.html?p...
2,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/0...,연청와이드핏-pt,"43,700won",https://www.lagirl.co.kr/product/detail.html?p...
3,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/d...,지브라밴딩-pt,"45,600won",https://www.lagirl.co.kr/product/detail.html?p...
4,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/4...,점프돌청-j/s,"27,600won",https://www.lagirl.co.kr/product/detail.html?p...
5,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/2...,하프앤데님-shorts,"25,700won",https://www.lagirl.co.kr/product/detail.html?p...
6,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/5...,플리츠밴딩-pt,"37,100won",https://www.lagirl.co.kr/product/detail.html?p...
7,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/2...,후드숏-set,"37,100won",https://www.lagirl.co.kr/product/detail.html?p...
8,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/c...,기본레더-sk,"35,000won",https://www.lagirl.co.kr/product/detail.html?p...
9,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/c...,하프앤숏-shorts,"42,000won",https://www.lagirl.co.kr/product/detail.html?p...


In [160]:
prds = []
index = []
names = []
for i in range(len(data2)) :
    prd = data2['link'][i]
    ind = data2.index[i]
    name = data2['name'][i]
    if prd not in prds : # 중복되는 상품이 없도록 url을 수집한다.
        prds.append(prd) 
        index.append(ind) # 해당 상품에 대한 인덱스 정보가 있어야 이후에 옷 이미지와 매칭시킬 수 있으므로 인덱스 정보까지 포함한다.
        names.append(name)

In [15]:
df2 = pd.DataFrame({"name":[],
                    "size" : [],
                    "허리" : [],
                    "힙" : [],
                    "밑위" : [],
                    "허벅지" : [],
                   "밑단" : [],
                   "트임" : [],
                   "총":[]})
prds = []
index = []
names = []
for i in range(len(data2)) :
    prd = data2['link'][i]
    ind = data2.index[i]
    name = data2['name'][i]
    if prd not in prds : # 중복되는 상품이 없도록 url을 수집한다.
        prds.append(prd) 
        index.append(ind) # 해당 상품에 대한 인덱스 정보가 있어야 이후에 옷 이미지와 매칭시킬 수 있으므로 인덱스 정보까지 포함한다.
        names.append(name)

for web in range(0,50):
    if 'set' not in names[web]:
        try:
            test_item = prds[web]
            req6 = requests.get(test_item)
            if req6 == 200 :
                soup6 = BeautifulSoup(req6.content, 'html.parser')
            else :
                req6 = requests.get(test_item, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
                soup6 = BeautifulSoup(req6.content, 'html.parser')

            size0 = soup6.find('div',{'class':'cont'}).find_all('p',{'style':['text-align: center;','text-align: center; ']})

            text0 = []
            for i in range(len(size0)):
                tt  = size0[i].text
                text0.append(tt)

            matchers = ['밑단','허벅지']

            matching = [s for s in text0 if any(xs in s for xs in matchers)]

            matching

            if '단위' in matching[0]: #if문사용 - 프리사이즈 - 해당 조건문 제대로 되는지 확인해야함 
                #프리사이즈가 데이터에 삽입되지가 않음...
                size_t = matching[0].split()
                size_nm = size_t[0]
                size_t2 = size_t[1:-1]

                size_col = ['size']
                size_row = [size_nm]
                for i in range(len(size_t2)):
                    match = re.match(r"([가-힣]+)([0-9].+)", size_t2[i], re.I)
                    if match:
                        items = match.groups()
                    size_col.append(items[0])
                    size_row.append(items[1])

                df_iframe = pd.DataFrame( columns = size_col)
                df = df_iframe.append(pd.Series(size_row, index=size_col), ignore_index=True)
                df["name"] = names[web]

                df2 = df2.append(df)
            if '단위' not in matching[0]:
                size_t = []
                for i in range(len(matching)):
                    size_t0 = matching[i].split()
                    size_t.append(size_t0)

                size_nm = []
                size_t2 = []
                for i in range(len(size_t)):
                    size_nm.append(size_t[i][0])
                    size_t2.append(size_t[i][1:])

                size_row2 = []
                size_col = ['size']
                for i in range(len(size_nm)):
                    size_row = [size_nm[i]]
                    for j in range(len(size_t2[0])):
                        match = re.match(r"([가-힣]+)([0-9].+)", size_t2[i][j], re.I)
                        if match:
                            items=match.groups()
                        size_row.append(items[1])
                        size_col.append(items[0])
                    size_row2.append(size_row)

                size_col2=size_col[0:len(size_t[0])]   

                data_f = pd.DataFrame(columns = size_col2)
                df_iframe = pd.DataFrame( columns = size_col2)
                for i in range(len(size_row2)):
                    df = df_iframe.append(pd.Series(size_row2[i], index=size_col2), ignore_index=True)
                    df["name"] = names[web]
                    data_f = data_f.append(df)

                df2 = df2.append(data_f)
        except:
            pass

df2["category"] = "Outer"
df2.index = range(len(df2))

C:\Users\tlsal\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [16]:
df2

,name,size,밑단,밑위,총,트임,허리,허벅지,힙,category
0,부츠라인데님-pt,S,23.5,25,97.5,NaN,32,22.5,38,Outer
1,부츠라인데님-pt,M,24.5,26,98.5,NaN,35,24.5,39,Outer
2,연청와이드핏-pt,S,20,29,97.5,NaN,32,30.5,47.5,Outer
3,연청와이드핏-pt,M,21,30,98.5,NaN,35,32.5,48.5,Outer
4,지브라밴딩-pt,(F44-66),33.5,37,104.5,NaN,28.5,32,39.5,Outer
5,하프앤숏-shorts,S,15,27,28/42.5,NaN,32,20,37,Outer
6,하프앤숏-shorts,M,16,28,29/43.5,NaN,35,22,38,Outer
7,흑청롤업숏-shorts,S,NaN,27,41,NaN,33.5,31.5,47.5,Outer
8,흑청롤업숏-shorts,M,NaN,28,42,NaN,36.5,33.5,48.5,Outer
9,흑청롤업숏-shorts,L,NaN,29,43,NaN,39.5,35.5,49.5,Outer


In [292]:
data2

,date,image,name,price,link
0,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/d...,부츠라인데님-pt,"46,600won",https://www.lagirl.co.kr/product/detail.html?p...
1,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/a...,비비미디-sk,"33,300won",https://www.lagirl.co.kr/product/detail.html?p...
2,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/0...,연청와이드핏-pt,"43,700won",https://www.lagirl.co.kr/product/detail.html?p...
3,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/d...,지브라밴딩-pt,"45,600won",https://www.lagirl.co.kr/product/detail.html?p...
4,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/4...,점프돌청-j/s,"27,600won",https://www.lagirl.co.kr/product/detail.html?p...
5,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/2...,하프앤데님-shorts,"25,700won",https://www.lagirl.co.kr/product/detail.html?p...
6,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/5...,플리츠밴딩-pt,"37,100won",https://www.lagirl.co.kr/product/detail.html?p...
7,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/2...,후드숏-set,"37,100won",https://www.lagirl.co.kr/product/detail.html?p...
8,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/c...,기본레더-sk,"35,000won",https://www.lagirl.co.kr/product/detail.html?p...
9,2019-03-21,//www.lagirl.co.kr/web/product/medium/201903/c...,하프앤숏-shorts,"42,000won",https://www.lagirl.co.kr/product/detail.html?p...


In [231]:
test_item = prds[4]
req6 = requests.get(test_item)
if req6 == 200 :
    soup6 = BeautifulSoup(req6.content, 'html.parser')
else :
    req6 = requests.get(test_item, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
    soup6 = BeautifulSoup(req6.content, 'html.parser')

size0 = soup6.find('div',{'class':'cont'}).find_all('p',{'style':['text-align: center;','text-align: center; ']})



In [18]:
names

['부츠라인데님-pt',
 '비비미디-sk',
 '연청와이드핏-pt',
 '지브라밴딩-pt',
 '점프돌청-j/s',
 '하프앤데님-shorts',
 '플리츠밴딩-pt',
 '후드숏-set',
 '기본레더-sk',
 '하프앤숏-shorts',
 '흑청롤업숏-shorts',
 '컬러부츠-pt',
 '빅데미지데님-pt',
 '도트플리-sk',
 '데미포인데님-pt',
 '레더바이크-leggings',
 '트임세미부츠-sl',
 '델리슬림트임-pt',
 '일자데미청-pt',
 '와이디롱-pt',
 '집업레깅배색-set',
 '비비롱-sk',
 '밀크워싱-pt',
 '트임부츠-sl',
 '브라이트하프-leggings',
 '플레이델리-set',
 '텐셔닝부츠-pt',
 '밑단절개조거-pt',
 '언발스포티-sk',
 '꽈배기니트-set',
 '밴딩더와이드-pt',
 '훼이포켓-pt',
 '컬러부츠컷-sl',
 '밴딩데님-pt',
 '티치티치-pt',
 '레트로풍-sk',
 '데미지루즈-pt',
 '모던일자-sl',
 '미니언발프릴-sk',
 '슬림워싱데미지-pt',
 '트레킹조거-pt',
 '컬러풀트임-sk',
 '핀턱부츠-pt',
 '델리델리-set',
 '데미지숏청-shorts',
 '숏버뮤-shorts',
 '델리부츠컷-sl',
 '데미와이-pt',
 '하프스키니-pt',
 '밴딩와이-pt',
 '어웨이버뮤-shorts',
 '엔나조거-pt',
 '밴딩롱트임-sk',
 '테니스주름-sk',
 '데일리집업조거-j/s',
 '와딩투스-sl',
 '오링코튼벨-pt',
 '밀크부츠-pt',
 '델리연청숏-shorts',
 '시보리델리-pt',
 '델리나코튼-pt',
 '미디유니콘-sk',
 '절개델리워싱-pt',
 '라인레더-pt',
 '화이트데님-pt',
 '하프클래식-pt',
 '델리시보리-pt',
 '레이스레더-sk',
 '슬림밴딩부츠컷-pt',
 '워싱수술숏-shorts',
 '하이스트-pt',
 '라인배색조거-pt',
 '연청데님일자-pt',
 '와이드데미지청

In [307]:
df2_json

,name,size,가슴,뒷트임,밑단,밑위,암홀(조절가능),어깨,총,트임,허리,허리(뒷밴딩),허리(밴딩),허리(조절가능),허벅지,힙
0,부츠라인데님-pt,S,None,None,23.5,25,None,None,97.5,None,32,None,None,None,22.5,38
0,부츠라인데님-pt,M,None,None,24.5,26,None,None,98.5,None,35,None,None,None,24.5,39
0,비비미디-sk,(F44-66),None,None,None,None,None,None,70,19.5,None,None,30,None,None,46
0,연청와이드핏-pt,S,None,None,20,29,None,None,97.5,None,32,None,None,None,30.5,47.5
0,연청와이드핏-pt,M,None,None,21,30,None,None,98.5,None,35,None,None,None,32.5,48.5
0,지브라밴딩-pt,(F44-66),None,None,33.5,37,None,None,104.5,None,28.5,None,None,None,32,39.5
0,점프돌청-j/s,(F44-66),43,None,20,None,42,28.5,130,None,None,None,None,None,None,45
0,플리츠밴딩-pt,(F44-66),None,None,25,31,None,None,95(단위:CM/단면),None,None,None,29,None,24,36
0,기본레더-sk,S,None,None,None,None,None,None,40,None,33,None,None,None,None,43
0,기본레더-sk,M,None,None,None,None,None,None,41,None,36,None,None,None,None,44


'[{"name":"\\ubd80\\uce20\\ub77c\\uc778\\ub370\\ub2d8-pt","size":"S","\\uac00\\uc2b4":null,"\\ub4b7\\ud2b8\\uc784":null,"\\ubc11\\ub2e8":"23.5","\\ubc11\\uc704":"25","\\uc554\\ud640(\\uc870\\uc808\\uac00\\ub2a5)":null,"\\uc5b4\\uae68":null,"\\ucd1d":"97.5","\\ud2b8\\uc784":null,"\\ud5c8\\ub9ac":"32","\\ud5c8\\ub9ac(\\ub4b7\\ubc34\\ub529)":null,"\\ud5c8\\ub9ac(\\ubc34\\ub529)":null,"\\ud5c8\\ub9ac(\\uc870\\uc808\\uac00\\ub2a5)":null,"\\ud5c8\\ubc85\\uc9c0":"22.5","\\ud799":"38"},{"name":"\\ubd80\\uce20\\ub77c\\uc778\\ub370\\ub2d8-pt","size":"M","\\uac00\\uc2b4":null,"\\ub4b7\\ud2b8\\uc784":null,"\\ubc11\\ub2e8":"24.5","\\ubc11\\uc704":"26","\\uc554\\ud640(\\uc870\\uc808\\uac00\\ub2a5)":null,"\\uc5b4\\uae68":null,"\\ucd1d":"98.5","\\ud2b8\\uc784":null,"\\ud5c8\\ub9ac":"35","\\ud5c8\\ub9ac(\\ub4b7\\ubc34\\ub529)":null,"\\ud5c8\\ub9ac(\\ubc34\\ub529)":null,"\\ud5c8\\ub9ac(\\uc870\\uc808\\uac00\\ub2a5)":null,"\\ud5c8\\ubc85\\uc9c0":"24.5","\\ud799":"39"},{"name":"\\ube44\\ube44\\ubbf8\\ub51